In [0]:
#Additional Packages to Install
# !pip install pydrive
# !pip install git+https://github.com/hyperopt/hyperopt.
# !pip install kopt
# !pip install oauth2client
# !pip install tensorboardcolab

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pydrive
from tensorboardcolab import *
from random import randint
from keras.layers import Dense,Conv1D,Input
from keras.layers import MaxPooling1D,UpSampling1D
from keras.layers import Activation,BatchNormalization
from keras.models import Model
from keras.optimizers import RMSprop
from sklearn.preprocessing import MinMaxScaler
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from kopt import CompileFN, KMongoTrials, test_fn
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials


#Authenticate/Create PyDrive client 
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Create File in Collab
#model_file = drive.CreateFile({'title' : 'backup.json'})
#model_file.SetContentFile('model.h5')

  
  


#tbc=TensorBoardColab()

batch_size = 1000
validation_split = 0.15
timesteps = 2000
features = 1
global timesteps
global features

def create_data():
  length = 1
  dt = 0.0005
  noise_factor = 0.35
  data = []
  data_noisy = []
  for i in range(batch_size):
      freq = 30*np.random.random() + 1
      t = np.arange(0,length,dt)
      #signal = np.arange(-0.5*length,0.5*length,dt)
      signal = np.sin(2*np.pi*freq*t)
      noisy_signal = signal+noise_factor*np.random.rand(len(t))
      data.append(signal)
      data_noisy.append(noisy_signal)
  data = np.array(data)
  data_noisy = np.array(data_noisy)
  
  #Scale Data - Need a better scaler than MinMaxScaler
  scaler = MinMaxScaler(feature_range = (0,1))
  data  = np.vstack((signal,signal_noisy))
  data = scaler.fit_transform(data)
  signal = data[0:batch_size][:]
  signal_noisy = data[batch_size:2*batch_size][:]
  
  #Input Shaping
  features = 1
  timesteps = len(signal[0])
  shape = (batch_size,timesteps,features)
  #Input shape should equal [samples, timesteps, features]
  x = signal.reshape(shape)
  y = signal_noisy.reshape(shape)
  
  x_train = x[0:(1-validation_split)*batch_size][:]
  y_train = y[0:(1-validation_split)*batch_size][:]
  x_test = x[(1-validation_split)*batch_size:batch_size][:]
  y_test = y[(1-validation_split)*batch_size:batch_size][:]

  return (x_train,y_train,features),(x_test,y_test)

#Create Test/Train Data




# #Prediction Data
# x_pred = signal_noisy[0][:]
# #Pre Autoencoder
# x_pred_pre = signal_noisy[0] 
# x_pred_pre = x_pred_pre.reshape((-1,len(x_pred_pre)))
# x_pred_pre = scaler.inverse_transform(x_pred_pre)
# #Clean
# clean_signal = signal[0][:]
# clean_signal = clean_signal.reshape((-1,len(clean_signal)))
# clean_signal = scaler.inverse_transform(clean_signal)



#For Optimization
def create_autoencoder(train_data,max_filters = 32, min_filters = 8, layers = 3,
                      kernel_size = 3,filter_spacing = 'lin'):
  
  timesteps = 2000
  features = 1
  inputs = Input(shape = (timesteps,features))
  encoded = inputs
  
  #Linear Filter Range
  if filter_spacing == 'lin':
    filter_range = np.linspace(max_filters,min_filters,layers,
                               dtype = 'int64')
  #Half Filter Range
  if filter_spacing == 'half':
    if max_filters%2 != 0:
      max_filters+=1
    if min_filters%2 != 0:
      min_filters+=1
    
    
    filter_range = [max_filters]
    x = max_filters
    while x> min_filters:
      x = x/2
      filter_range.append(x)
    if filter_range[-1] < min_filters:
      del filter_range[-1]
    filter_range = np.asarray(filter_range,dtype = 'int64')
  
  #Random Filter Range
  if filter_spacing == 'random':
    filter_range = [randint(min_filters, max_filters) for z in range(layers)]
    filter_range.sort(reverse= True)
    filter_range = np.asarray(filter_range)
    
  
  for num_filters in filter_range: 
    encoded = Conv1D(num_filters,kernel_size,padding = 'same')(encoded)
  
    if(num_filters==max_filters):
      encoded = BatchNormalization()(encoded)
  
    encoded = Activation('relu')(encoded)
    encoded = MaxPooling1D(2,padding = 'same')(encoded)
  
  filter_range = np.sort(filter_range)
  for num_filters in filter_range: 
    encoded = Conv1D(num_filters,kernel_size,padding = 'same')(encoded)
  
    encoded = Activation('relu')(encoded)
    encoded = UpSampling1D(2)(encoded)
    
  decoded = Conv1D(1,kernel_size,activation = 'sigmoid',padding = 'same')(encoded)
  
  autoencoder = Model(inputs,decoded)
  
  return autoencoder


#NOT THE OPTIMIZED MODEL
def create_Conv1D():
  inputs = Input(shape = (timesteps,features))
  
  encoded = Conv1D(32,3, padding = 'same')(inputs)
  encoded = BatchNormalization()(encoded)
  encoded = Activation('relu')(encoded)
  encoded = MaxPooling1D(2,padding = 'same')(encoded)
  
  
  encoded = Conv1D(16,3,padding = 'same')(encoded)
  encoded = Activation('relu')(encoded)
  encoded = MaxPooling1D(2,padding = 'same')(encoded)
  
  encoded = Conv1D(8,3, padding = 'same')(encoded)
  encoded = Activation('relu')(encoded)
  encoded = MaxPooling1D(2,padding = 'same')(encoded)
  
  
  decoded = Conv1D(8,3,padding = 'same')(encoded)
  decoded = Activation('relu')(decoded)
  decoded = UpSampling1D(2)(decoded)
  
  decoded = Conv1D(16,3, padding = 'same')(decoded)
  decoded = Activation('relu')(decoded)
  decoded = UpSampling1D(2)(decoded)
  
  decoded = Conv1D(32,3,padding = 'same')(decoded)
  decoded = Activation('relu')(decoded)
  decoded = UpSampling1D(2)(decoded)
  
  decoded = Conv1D(1,3,activation = 'sigmoid',padding = 'same')(decoded)
  
  autoencoder = Model(inputs,decoded)
  
  optimizer = RMSprop(lr=1e-3)
  autoencoder.compile(optimizer = optimizer, loss = 'mse')
  
  return autoencoder





def nn_predict(model,prediction_data):
  pad_length = timesteps-(np.size(prediction_data)%timesteps)
  if pad_length != timesteps:
    prediction_data = np.pad(prediction_data,[(0,0),(0,pad_length)],
                             mode = 'constant', constant_values = 0)
   
  prediction_data = prediction_data.reshape((-1,timesteps))
  
  predicted_data = []
  
  for i in range(np.size(prediction_data,0)):
    pred_part = prediction_data[i].reshape(1,timesteps,features) 
    pred_part = model.predict(pred_part)
    predicted_data.append(pred_part)
  
  predicted_data = (np.array(predicted_data)).flatten()
  
  predicted_data = predicted_data.reshape(-1,len(predicted_data))
  predicted_data = scaler.inverse_transform(predicted_data)
  return predicted_data


#HyperParameter Range
hyper_parms = {
    "model":{
        "max_filters":hp.randint('mx_filters',(32,320))
        "min_filters":hp.randint('mn_filters',(4,32))
        'layers':hp.randint('ly',(3,12))
        'filter_spacing':hp.choice('fs'['lin','half','random'])
        
    }
    "fit":{
        'epochs': 150,
        'patience': 3
    }
}



#NN Evaluation
db_name = 'sine'
exp_name ='exp_1'
objective = CompileFN(db_name,exp_name,
                      data_fn = create_data,
                      model_fm = create_autoencoder,
                      loss_metric = 'loss',
                      loss_metric_mode = 'min'
                      valid_split = None,
                      save_model = 'best')
#                     save_results = True,
#                     save_dir = model_file.SetContentFile('model.h5')
                      


# model_file.Upload()
# drive.CreateFile({'id': model_file.get('id')})

# test_fn(objective, hyper_params)
# trials = Trials()
# best = fmin(objective, hyper_params, trials=trials, algo=tpe.suggest, max_evals=2)
# trials = KMongoTrials(db_name, exp_name,
#                       ip="localhost",
# 	              port=22334)
# best = fmin(objective, hyper_params, trials=trials, algo=tpe.suggest, max_evals=2)






#NN HyperParameters
# epochs = 1

# inputs = 1

# #NN Fitting
# autoencoder = create_autoencoder(inputs,filter_spacing = 'lin')
# autoencoder_correct = create_Conv1D()
# autoencoder_correct.summary()
# autoencoder.summary()




#Prediction Data
#prediction = nn_predict(autoencoder,x_pred).flatten()

         
# #Plot Prediction                                   
# fig = plt.figure()
# ax = plt.subplot(111)
# ax.plot(t,prediction,label = 'Denoised Signal')
# ax.plot(t,clean_signal.flatten(), label = 'Clean Signal')
# ax.plot(t,x_pred_pre.flatten(), label = 'Noisy Signal')
# plt.xlim(0,0.25)
# ax.legend(loc = 3)
  
  
#Save Model to Google Drive
# autoencoder.save('model.h5', include_optimizer=False)
# model_file = drive.CreateFile({'title' : 'CNN1D_v1.0.h5'})
# model_file.SetContentFile('model.h5')
# model_file.Upload()
# drive.CreateFile({'id': model_file.get('id')})
  
  
  
  
  
  

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 2000, 1)           0         
_________________________________________________________________
conv1d_36 (Conv1D)           (None, 2000, 32)          128       
_________________________________________________________________
batch_normalization_6 (Batch (None, 2000, 32)          128       
_________________________________________________________________
activation_31 (Activation)   (None, 2000, 32)          0         
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, 1000, 32)          0         
_________________________________________________________________
conv1d_37 (Conv1D)           (None, 1000, 16)          1552      
_________________________________________________________________
activation_32 (Activation)   (None, 1000, 16)          0         
__________